# Scalable Supervised Machine Learning on the ODC

This notebook is a placeholder for now.  It will contain a markdown only description of the scripts in this folder, including:
- The guiding principles/philosophy of the scripts (ie setting expectations around what they can and cannot do)
- The order to run the scripts in (including links to each script)
- Any prerequisites required to understand the content in these notebooks (e.g. reading material, ODC/Sandbox/notebooks/python training material)
- Information on any set-up that might be required to run these notebooks (e.g. creating folders for outputs? Setting up a S3 bucket for intermediate files?)


---
## TO DO
- add a `debug=True|False` flag to `collect_training_data`, this will set ncpus=1 and remove HiddenPrints()
- add WOfS loading option to `collect_training_data`?
- implement learning curves? https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
- rewrite the `sklearn_(un)flatten` to work with `dask.arrays`??
- convert .GeoMedian() to xr_geomedian in classificationtools for consistency
- check if we can use `sklearn_unflatten` to return the features from `predict_xr`